# NLP with oil and renewable energies project

#### Get libraries

In [24]:
import requests
import re
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
import numpy as np
from PIL import Image
from sklearn.linear_model import LogisticRegression
import argparse
from matplotlib import pyplot as plt
import wordcloud
import requests, json
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns

In [25]:
corpus_oil_price=pd.read_csv("data/oil_price.csv", index_col=0)
corpus_oil_price["date_published"] = pd.to_datetime(corpus_oil_price["date_published"], errors='coerce', utc=True).dt.to_period('m')
corpus_oil_price

C:\Users\kaypa\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,title,date_published,content
0,$70 Oil Won't Keep OPEC Happy,2021-11,One day&mdash;that was all it took oil prices ...
1,Why This Oil Price Slump May Not Be Bad News F...,2021-12,Crude oil prices are plunging under the weight...
2,The Double Irony Of Biden's Plea To OPEC,2021-11,As I wrote in February of this year in&nbsp;Th...
3,The Oil Price Rally Is Far From Over,2021-11,How much higher could oil prices go This is th...
4,"Biden Threatens OPEC With Undisclosed ""Tools""",2021-11,"""There are other tools in the arsenal that we ..."
...,...,...,...
2183,A Disobliging Update on the Price of Oil,2010-01,There still seems to be a deplorable uncertain...
2184,What Happens When the Wells Run Dry,2010-01,One nagging question that the industrial world...
2185,Petro Unfriendly California Changing it's Stan...,2009-12,The financial crisis changed the way people th...
2186,New Techniques Oil Companies are Using in Dril...,2009-11,As the politics and philosophical arguments ab...


In [26]:
a = corpus_oil_price.groupby('date_published')[['title']].count()
a['date'] = a.index
a.to_csv("data/oilprice.csv")

In [27]:
corpus_alternative_energy=pd.read_csv("data/alternative_energy.csv", index_col=0)
corpus_alternative_energy["date_published"] = pd.to_datetime(corpus_alternative_energy["date_published"], errors='coerce', utc=True).dt.to_period('m')

C:\Users\kaypa\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [28]:
b = corpus_alternative_energy.groupby('date_published')[['title']].count()
b['date'] = b.index
b.to_csv("data/alternative.csv")
b

,title,date
date_published,,
2009-07,6,2009-07
2009-09,2,2009-09
2009-10,2,2009-10
2009-11,2,2009-11
2010-02,5,2010-02
...,...,...
2021-07,12,2021-07
2021-08,13,2021-08
2021-09,7,2021-09


In [29]:
corpus_crude_oil=pd.read_csv("data/crude_oil_df.csv", index_col=0)
corpus_crude_oil["date_published"] = pd.to_datetime(corpus_crude_oil["date_published"], errors='coerce', utc=True).dt.to_period('m')

C:\Users\kaypa\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [30]:
c = corpus_crude_oil.groupby('date_published')[['title']].count()
c['date'] = c.index
c.to_csv("data/crudeoil.csv")

In [31]:
corpus_oil_price2 = pd.read_csv('data/corpus_oil_price_3.csv')
corpus_alternative_energies2 = pd.read_csv('data/corpus_alternative_energies_3.csv')
corpus_crude_oil2 = pd.read_csv('data/corpus_crude_oil_3.csv')
corpus_crude_oil2

,Unnamed: 0,title,date_published,content,baseline_sentiment,vader_sentiment,most_common_words
0,0,Oil Bulls Rejoice As Biden's Supply Strategy B...,2021-11-24T18:00:00-06:00,President Bidenrsquos threat to oil producers ...,-1,-0.9926,"['oil', 'prices', 'OPEC', 'Bidenrsquos', 'rele..."
1,1,"OPEC To Add 400,000 Bpd In January Despite Oil...",2021-12-02T08:53:55-06:00,The OPEC group is sticking to its plan to ease...,0,0.4497,"[' ', 'OPEC', 'bpd', 'January', '400,000', 'me..."
2,2,UAE Pumps $6 Billion Into Oil And Gas Expansio...,2021-11-24T17:00:00-06:00,The U.S.-sponsored lsquorelationship normalisa...,1,0.9875,"['oil', 'UAErsquos', 'billion', 'UAE', 'US$', ..."
3,3,China Keeps Markets In The Dark About SPR Release,2021-11-24T16:00:00-06:00,The volume of the expected Chinese release of ...,-1,-0.9552,"['oil', ' ', 'release', 'crude', 'reserves', '..."
4,4,Oil Markets Unimpressed By Small Crude Invento...,2021-11-24T09:36:00-06:00,"Crude oil inventories rose last week, while ga...",-1,-0.9601,"['million', ' ', 'barrels', 'week', 'bpd', 'cr..."
...,...,...,...,...,...,...,...
5000,5000,A Detailed Guide on the Many Different Types o...,2009-12-02T17:12:01-06:00,Some people arbitrarily speak about oil as if ...,-1,-0.9962,"['oil', 'crude', 'oils', 'WTI', 'Oils', 'Brent..."
5001,5001,What is Peak Oil Theory A Thorough Look at Thi...,2009-10-21T16:17:44-05:00,Currently there is a lot of debate going on re...,1,-0.1567,"['oil', 'time', 'world', 'supply', 'production..."
5002,5002,Oil Shale - So Just What Is It,2009-09-24T17:26:55-05:00,People often say ldquoYou canrsquot squeeze bl...,-1,0.9816,"['oil', 'shale', 'energy', 'process', 'rock', ..."
5003,5003,Oil is not a Villain Here's what it's done for us,2009-09-21T16:45:02-05:00,"For decades now, oil has often been vilified a...",1,0.9987,"['oil', 'energy', 'world', 'time', 'Fact', 'ye..."


In [32]:
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("[","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("]","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("'","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace("',","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace(",'","")
corpus_oil_price2['most_common_words'] = corpus_oil_price2['most_common_words'].str.replace(",","")

In [33]:
col_one_list_2 = corpus_oil_price2['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

[['oil',
  '',
  '',
  'OPEC',
  '$',
  'release',
  'new',
  'bpd',
  'million',
  'barrels',
  'rdquo',
  'prices',
  'latest',
  'variant',
  'output',
  'SPR',
  'global',
  'market',
  'U.S.',
  'halt',
  'supply',
  'demand'],
 ['',
  '',
  'production',
  'shale',
  'oil',
  'prices',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest',
  'variant',
  'selloff',
  'U.S.',
  'stocks',
  'patch',
  'discipline',
  'cash',
  'disciplined',
  'said',
  'depletion'],
 ['oil',
  '',
  '',
  'Biden',
  'prices',
  'Administration',
  'production',
  'mdash',
  'U.S.',
  '',
  '',
  '',
  '',
  'year',
  'fossil',
  'people',
  'pump',
  'President',
  'higher',
  'policies',
  'supplies',
  'meeting',
  'demand',
  'week',
  'OPEC'],
 ['oil',
  '',
  '',
  'prices',
  'demand',
  '$',
  'higher',
  'barrel',
  'year',
  'said',
  'supply',
  'bullish',
  'remains',
  'U.S.',
  'rdquo',
  '90',
  'month',
  'OPEC',
  'price',
  'view',
  'note',
  'Brent'],
 ['',

In [34]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil"
            
            
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['',
  'OPEC',
  'release',
  'new',
  'million',
  'rdquo',
  'latest',
  'variant',
  'output',
  'SPR',
  'global',
  'market',
  'U.S.',
  'halt',
  'supply',
  'demand'],
 ['',
  'production',
  'shale',
  'prices',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest',
  'variant',
  'selloff',
  'U.S.',
  'stocks',
  'patch',
  'discipline',
  'cash',
  'disciplined',
  'depletion'],
 ['Biden',
  'Administration',
  'production',
  'mdash',
  'U.S.',
  '',
  '',
  '',
  'fossil',
  'people',
  'pump',
  'President',
  'higher',
  'policies',
  'supplies',
  'meeting',
  'demand',
  'OPEC'],
 ['',
  'prices',
  'demand',
  'higher',
  'year',
  'supply',
  'bullish',
  'remains',
  'U.S.',
  '90',
  'month',
  'OPEC',
  'view',
  'note',
  'Brent'],
 ['',
  'OPEC',
  'pump',
  'tools',
  'President',
  'Biden',
  'crude',
  'Washington',
  'global',
  'recovery',
  'arsenal',
  'countries',
  'supply',
  'response',
  'calls',
  'respond',
  'quoted',
  'ad

In [35]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil"]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['OPEC',
  'release',
  'new',
  'rdquo',
  'latest',
  'variant',
  'output',
  'SPR',
  'global',
  'market',
  'U.S.',
  'halt',
  'supply',
  'demand'],
 ['production',
  'shale',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest',
  'variant',
  'selloff',
  'U.S.',
  'stocks',
  'patch',
  'discipline',
  'cash',
  'disciplined',
  'depletion'],
 ['Biden',
  'Administration',
  'production',
  'mdash',
  'U.S.',
  '',
  'fossil',
  'people',
  'pump',
  'President',
  'higher',
  'policies',
  'supplies',
  'meeting',
  'demand',
  'OPEC'],
 ['prices',
  'demand',
  'higher',
  'supply',
  'bullish',
  'remains',
  'U.S.',
  '90',
  'month',
  'OPEC',
  'view',
  'note',
  'Brent'],
 ['OPEC',
  'pump',
  'tools',
  'President',
  'Biden',
  'Washington',
  'global',
  'recovery',
  'arsenal',
  'countries',
  'supply',
  'response',
  'calls',
  'respond',
  'quoted',
  'administration'],
 ['OPEC',
  'Morse',
  'production',
  'prices',
  'U.S.',
  'capa

In [36]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df_oil_price = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
word_cnt_df_oil_price.sort_values(by='cnt', ascending=False, inplace=True)
word_cnt_df_oil_price = word_cnt_df_oil_price.nlargest(30, 'cnt')
word_cnt_df_oil_price.to_csv('data/word_cnt_df_oil_price.csv')

In [37]:
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("[","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("]","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("'","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace("',","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace(",'","")
corpus_alternative_energies2['most_common_words'] = corpus_alternative_energies2['most_common_words'].str.replace(",","")

In [38]:
col_one_list_2 = corpus_alternative_energies2['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

[['',
  '',
  'energy',
  'tons',
  'metals',
  '$',
  'costs',
  'battery',
  'prices',
  'renewable',
  'copper',
  'big',
  'solar',
  'transition',
  'new',
  'estimates',
  'global',
  'trillion',
  'BNEF',
  '2030',
  'EV',
  'green'],
 ['',
  '',
  'report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  '$',
  'Net',
  'Zero',
  'rdquo',
  'energy',
  'said',
  'public',
  'Asia',
  'governments',
  'Global',
  'capital',
  'collective'],
 ['energy',
  'Europe',
  '',
  '',
  'prices',
  'gas',
  'European',
  'transition',
  'high',
  'said',
  'EU',
  'plans',
  'Birol',
  'sky',
  'Russia',
  'send',
  'additional',
  'volumes',
  'Gazprom',
  'market',
  'storage',
  'coal'],
 ['energy',
  '',
  '',
  'global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world',
  'oil',
  'going',
  'lithium',
  'supply',
  'decades',
  'energies',
  'batteries',
  'infrastructure',
  'battery',
  'nations'

In [39]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power"           
            
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['',
  'tons',
  'metals',
  'costs',
  'battery',
  'renewable',
  'copper',
  'big',
  'solar',
  'transition',
  'new',
  'estimates',
  'global',
  'trillion',
  'BNEF',
  '2030',
  'EV',
  'green'],
 ['',
  'report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  'Net',
  'Zero',
  'energy',
  'public',
  'Asia',
  'governments',
  'Global',
  'capital',
  'collective'],
 ['Europe',
  '',
  'gas',
  'European',
  'transition',
  'high',
  'EU',
  'plans',
  'Birol',
  'sky',
  'Russia',
  'send',
  'additional',
  'volumes',
  'Gazprom',
  'market',
  'storage',
  'coal'],
 ['',
  'global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world',
  'going',
  'lithium',
  'supply',
  'decades',
  'energies',
  'batteries',
  'infrastructure',
  'battery',
  'nations',
  'industry'],
 ['',
  'carbon',
  'solar',
  'Chevron',
  'wind',
  'power',
  'gas',
  'renewable',
  'emissions',
  'CCS',
  'Exxon',

In [40]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power"           
            
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['tons',
  'metals',
  'costs',
  'battery',
  'renewable',
  'copper',
  'big',
  'solar',
  'transition',
  'new',
  'estimates',
  'global',
  'trillion',
  'BNEF',
  '2030',
  'EV',
  'green'],
 ['report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  'Net',
  'Zero',
  'public',
  'Asia',
  'governments',
  'Global',
  'capital',
  'collective'],
 ['Europe',
  'gas',
  'European',
  'transition',
  'high',
  'EU',
  'plans',
  'Birol',
  'sky',
  'Russia',
  'send',
  'additional',
  'volumes',
  'Gazprom',
  'market',
  'storage',
  'coal'],
 ['global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world',
  'going',
  'lithium',
  'supply',
  'decades',
  'energies',
  'batteries',
  'infrastructure',
  'battery',
  'nations',
  'industry'],
 ['carbon',
  'solar',
  'Chevron',
  'wind',
  'gas',
  'renewable',
  'emissions',
  'CCS',
  'Exxon',
  'low',
  'lower',
  'plans',
  'invest',
  'busine

In [41]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df_renewables = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
word_cnt_df_renewables.sort_values(by='cnt', ascending=False, inplace=True)
word_cnt_df_renewables = word_cnt_df_renewables.nlargest(30, 'cnt')
word_cnt_df_renewables.to_csv('data/word_cnt_df_renewables.csv')

In [42]:
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("[","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("]","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("'","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace("',","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace(",'","")
corpus_crude_oil2['most_common_words'] = corpus_crude_oil2['most_common_words'].str.replace(",","")

In [43]:
col_one_list_2 = corpus_crude_oil2['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

[['oil',
  'prices',
  'OPEC',
  'Bidenrsquos',
  'release',
  'crude',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market',
  'China',
  'production',
  'Saudi',
  'demand',
  'energy',
  'U.S.',
  'increase',
  'push',
  'price',
  'strategy'],
 ['',
  '',
  'OPEC',
  'bpd',
  'January',
  '400000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy',
  'sources',
  'said',
  'Russia',
  'need',
  'oil',
  'surplus',
  'alliance',
  'ministers',
  'Bakr',
  'Deputy'],
 ['oil',
  'UAErsquos',
  'billion',
  'UAE',
  'US$',
  'key',
  'million',
  'deal',
  'Abu',
  'Dhabi',
  'ADNOC',
  'crude',
  'U.S.',
  'ndash',
  'Israel',
  '',
  '',
  'Murban',
  'bpd',
  'ICE',
  'East',
  'announced'],
 ['oil',
  '',
  '',
  'release',
  'crude',
  'reserves',
  'China',
  'prices',
  'said',
  'Chinese',
  'releasing',
  'coordinated',
  'Reuters',
  'consuming',
  'nations',
  'major',
  'strategic',
  'state',
  'according',
  'country',
  'actual',
  'needs'],

In [44]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power",      
            "null", "oil", "barrels", "$", "million", "bpd", "crude", "years", "like"
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['prices',
  'OPEC',
  'Bidenrsquos',
  'release',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market',
  'China',
  'production',
  'Saudi',
  'demand',
  'U.S.',
  'increase',
  'push',
  'strategy'],
 ['',
  'OPEC',
  'January',
  '400000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy',
  'sources',
  'Russia',
  'need',
  'surplus',
  'alliance',
  'ministers',
  'Bakr',
  'Deputy'],
 ['UAErsquos',
  'UAE',
  'US$',
  'key',
  'deal',
  'Abu',
  'Dhabi',
  'ADNOC',
  'U.S.',
  'Israel',
  '',
  'Murban',
  'ICE',
  'East',
  'announced'],
 ['',
  'release',
  'reserves',
  'China',
  'said',
  'Chinese',
  'releasing',
  'coordinated',
  'Reuters',
  'consuming',
  'nations',
  'major',
  'strategic',
  'state',
  'according',
  'country',
  'actual',
  'needs'],
 ['',
  'barrels',
  'bpd',
  'draw',
  'oil',
  'inventories',
  'previous',
  'gasoline',
  'EIA',
  'reported',
  'November',
  '19',
  'compared',
  'production',
  'data',
  'releas

In [45]:
stopwords = ["$","d","oil","prices","price",""," ", "year", "week", "barrel", "million", "percent", "said", "bpd", 
             "barrels",
            "oil", "crude", "ndash", "rdquo", "day", "Crude", "Oil", "Energy", "billion", "energy", "power",      
            "null", "oil", "barrels", "$", "million", "bpd", "crude", "years", "like"
            ]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

[['OPEC',
  'Bidenrsquos',
  'release',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market',
  'China',
  'production',
  'Saudi',
  'demand',
  'U.S.',
  'increase',
  'push',
  'strategy'],
 ['OPEC',
  'January',
  '400000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy',
  'sources',
  'Russia',
  'need',
  'surplus',
  'alliance',
  'ministers',
  'Bakr',
  'Deputy'],
 ['UAErsquos',
  'UAE',
  'US$',
  'key',
  'deal',
  'Abu',
  'Dhabi',
  'ADNOC',
  'U.S.',
  'Israel',
  'Murban',
  'ICE',
  'East',
  'announced'],
 ['release',
  'reserves',
  'China',
  'Chinese',
  'releasing',
  'coordinated',
  'Reuters',
  'consuming',
  'nations',
  'major',
  'strategic',
  'state',
  'according',
  'country',
  'actual',
  'needs'],
 ['barrels',
  'draw',
  'inventories',
  'previous',
  'gasoline',
  'EIA',
  'reported',
  'November',
  '19',
  'compared',
  'production',
  'data',
  'release',
  'trading'],
 ['release',
  'reserve',
  'OPEC',
  'accordi

In [46]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df_crude = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
word_cnt_df_crude.sort_values(by='cnt', ascending=False, inplace=True)
word_cnt_df_crude = word_cnt_df_crude.nlargest(30, 'cnt')
word_cnt_df_crude.to_csv('data/word_cnt_df_crude.csv')